# Whisper-tiny on IPU

This notebook demonstrates inference with Whisper-tiny on IPU using FP16.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2023 Graphcore Ltd. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

""" Run inference on a 🤗 Whisper model """

from optimum.utils import logging
from dataclasses import dataclass, field
from pathlib import Path

import torch
from datasets import load_dataset, Dataset
from tqdm import tqdm

import numpy as np

import poptorch
from optimum.graphcore import IPUConfig, IPUTrainer,IPUTrainingArguments
from optimum.graphcore.modeling_utils import to_pipelined
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

/localdata/paolot/popsdk-venvs/poplar_sdk-ubuntu_20_04-3.1.0+1205-58b501c780/3.1.0+1205_poptorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperConfig

In [4]:
from dataclasses import dataclass
from typing import List

@dataclass
class IPUWhisperConf:
    """A data class to collect IPU-related config parameters"""
    model_spec: str
    layers_per_ipu: List
    pod_type: str

ipu_whisper = {
    "tiny": IPUWhisperConf(model_spec='openai/whisper-tiny.en', layers_per_ipu=[8], pod_type="pod4"),
    "small": IPUWhisperConf(model_spec='openai/whisper-small', layers_per_ipu=[6,6,6,6], pod_type="pod4"),
    "large": IPUWhisperConf(model_spec='openai/whisper-large-v2', layers_per_ipu=[4,4,4,4,4,4,4,4, 4,4,4,4,4,4,4,4], pod_type="pod16")
}


In [5]:
model_size = "tiny"
iwc = ipu_whisper[model_size]

In [6]:
# Instantiate processor and model
processor = WhisperProcessor.from_pretrained(iwc.model_spec)
model = WhisperForConditionalGeneration.from_pretrained(iwc.model_spec)

In [7]:
# load dummy dataset and read soundfiles
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
input_features = processor(ds[0]["audio"]["array"], 
                           return_tensors="pt",
                           sampling_rate=ds[0]['audio']['sampling_rate']).input_features

Found cached dataset librispeech_asr_dummy (/home/paolot/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [8]:
import os
from pathlib import Path

pod_type = os.getenv("GRAPHCORE_POD_TYPE", iwc.pod_type)
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/whisper_exe_cache/") + "whisper_inference"

In [9]:
ipu_config = IPUConfig(executable_cache_dir=executable_cache_dir, layers_per_ipu=iwc.layers_per_ipu)

In [10]:
pipelined_model = to_pipelined(model, ipu_config)

In [11]:
pipelined_model = pipelined_model.parallelize().half()

In [12]:
sample_output = pipelined_model.generate(input_features, max_length=448, min_length=3)
transcription = processor.batch_decode(sample_output, skip_special_tokens=False)[0]
transcription

Graph compilation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00]


'<|startoftranscript|><|notimestamps|> Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.<|endoftext|>'